# Recupero para Bienes de Capital

INPUTS:
* Armar en downloads la carpeta con los archivos recibidos para correr el reproceso llamada: 'Bienes_MMMMYYYY'
* Debe contener:
    * Bienes.mdb (Contiene el despiece de piezas por factura)
    * Piezas MMMMYYYY.xlsx (Contiene las facturas de mes correspondiente y el Vin)
    * Rubro 3 MMYYYY.xlsx (Contiene el desglose de movimientos para el mes -sirve de comparativo-)
* *MMMM corresponde al mes completo (ejemplo Enero) y MM al mes en numero (ejemplo 01)*
* *YYYY corresponde al año completo (ejemplo 2017)*

In [1]:
############# Librerias #############
import pandas as pd
import shutil
import os.path
import numpy as np
from time import time
start_time = time()

In [2]:
def eliminar_bienes(Mes,Año):
    shutil.rmtree('C:/Volkswagen/Año de recupero '+Año+'/Bienes '+Mes)

In [3]:
eliminar_bienes('Febrero','2017')

In [4]:
def Bienes (Mes, Año):
    if os.path.exists(r'C:\Users\llloveras002\Downloads\Bienes_'+Mes+Año):
        if os.path.exists('C:/Volkswagen/Año de recupero '+Año+'/Bienes '+Mes):
            print('''Ya existe el directorio 'Bienes %s' en la ruta C:/Volkswagen/Año de recupero %s''' %(Mes, Año))
        else:
            shutil.copytree('C:/Volkswagen/Estructura Bienes',
                'C:/Volkswagen/Año de recupero '+Año+'/Bienes '+Mes)
            shutil.copytree(r'C:\Users\llloveras002\Downloads\Bienes_'+Mes+Año,
                'C:/Volkswagen/Año de recupero '+Año+'/Bienes '+Mes+'/01 Piezas/01 Archivos a utilizar')
    else:
        print('''El directorio 'Bienes_%s%s' en downloads, no existe.''' %(Mes, Año))

Funcion Bienes(Mes, Año):
* Parametros:
    * Mes: formato completo MMMM en texto (ejemplo 'Enero')
    * Año: formato YYYY en texto (ejemplo '2017')
* Pasos:
    * Comprueba que exista el directorio 'Bienes_MMMMYYYY' en downloads
    * Comprueba que exista o crea la carpeta 'Bienes MMMM' en el año correspondiente (Estructura Bienes) 
    * Copia el directorio 'Bienes_MMMMYYYY' en la estructura Volkswagen - Año de recupero YYYY - Bienes MMMM

In [5]:
Bienes('Febrero','2017')

## Exportacion o ventas nacionales

In [6]:
Año= '2017'
Mes= 'Febrero'

In [7]:
# Funcion para reemplazar el sistema decimal de , a .
def to_float_from_str_decimal(string):
    # It convert '12.540,5' to 125040.50
    try:
        return float(string.replace('.', '').replace(',', '.'))
    except:
        return string

In [8]:
# Import factura (lo que entregan los de impuestos Price)
factura = pd.read_excel('C:/Volkswagen/Año de recupero '+Año+'/Bienes '+Mes+'/01 Piezas/01 Archivos a utilizar/Facturas '+Mes+Año+'.xlsx')
# Igualar nombre de clave 'FACTURA'
factura = factura.rename(columns = {'Factura':'FACTURA'})
print("Cantidad de Facturas emitidas en el mes: " + str(len(factura)))

Cantidad de Facturas emitidas en el mes: 1008


In [9]:
# Comprobar que no hay duplicados en Factura (Control interno para los joins)
factura[factura.duplicated(['FACTURA'])]

,VIN,Right_Modelo,MES 2,FACTURA


In [10]:
# Import despiece por factura (lo que etrega la compañia VW)
despiece_factura = pd.read_csv('C:/Volkswagen/Año de recupero '+Año+'/Bienes '+Mes+'/01 Piezas/01 Archivos a utilizar/Despieces por Factura.txt', 
                       sep=';', encoding = 'unicode_escape', converters={'CANTIDAD': to_float_from_str_decimal})
# Convertir nulos en ''
despiece_factura= despiece_factura.fillna('')
print('La cantidad de piezas utilizadas en el proceso son: '+str(len(despiece_factura)))

La cantidad de piezas utilizadas en el proceso son: 1323583


In [11]:
# Join de Factura y despiece por factura 
merge_factura_despiece = pd.merge(left=despiece_factura, right=factura[['FACTURA']], how='inner', left_on='FACTURA', right_on='FACTURA')
# Agrupar por PBS y Cantidad
agrupado_piezas = merge_factura_despiece.groupby(['PREFIJO', 'BASICO', 'SUFIJO', 'PIEC_DESCRIPTION'])['CANTIDAD'].sum().reset_index()

In [12]:
#Prueba para comparar que la tabla de despiece_piezas con agrupado_piezas son iguales (control interno para saber si la agrupacion fue correcta)
despiece_piezas = pd.read_csv('C:/Volkswagen/Año de recupero '+Año+'/Bienes '+Mes+'/01 Piezas/01 Archivos a utilizar/Despieces por Pieza.txt', 
                       sep=';', encoding = 'unicode_escape', converters={'CANTIDAD': to_float_from_str_decimal})
despiece_piezas = despiece_piezas.fillna('')
# Debe dar Cero
print (sum(despiece_piezas['CANTIDAD'])/len(factura)-sum(agrupado_piezas['CANTIDAD'])/len(factura))

0.0


### Sacar del Universo de piezas las RAF utilizando equivalencias

In [13]:
#Importar despieces_RAF
despiece_RAF = pd.read_csv('C:/Volkswagen/Año de recupero '+Año+'/Bienes '+Mes+'/01 Piezas/01 Archivos a utilizar/Piezas Descargadas RAF.txt', 
                       sep=';', encoding = 'unicode_escape', low_memory=False, converters={'CANTIDAD': to_float_from_str_decimal})
#Convertir los NaN en vacios
despiece_RAF = despiece_RAF.fillna('')
#Colocar la Cantidad en negativo para sacarlos del universo ya que son importaciones
despiece_RAF['CANTIDAD'] = despiece_RAF['CANTIDAD']*(-1.00)
# Join de Factura y despiece por RAF 
merge_RAF_despiece = pd.merge(left=despiece_RAF, right=factura[['FACTURA']], how='inner', left_on='FACTURA', right_on='FACTURA')
# Agrupar por PBS y Cantidad
agrupado_piezas_RAF = merge_RAF_despiece.groupby(['PREFIJO', 'BASICO', 'SUFIJO', 'PIEC_DESCRIPTION'])['CANTIDAD'].sum().reset_index()

In [71]:
agrupado_piezas_RAF

,PREFIJO,BASICO,SUFIJO,PIEC_DESCRIPTION,CANTIDAD
0,022,103534,A,"CUBIERTA, DE PLASTICO",-1882.0
1,038,903315,AM,,-957.0
2,03C,906051,F,,-957.0
3,03G,145276,,POLEA LISA CON RODAMIENTO INTERIOR,-914.0
4,03G,145291,,CAPUCHON DE CIERRE,-941.0
...,...,...,...,...,...
1216,WHT,005599,,TORNILLO,-734.0
1217,WHT,006148,,Tornillo Combinado,-3696.0
1218,WHT,006152,,Tornillo Combinado,-4.0
1219,WHT,007077,,,-5736.0


In [14]:
#Concatenar los universo de Factura y RAF
piezas_sin_agrupar = pd.concat([agrupado_piezas[['SUFIJO','PREFIJO','PIEC_DESCRIPTION','CANTIDAD','BASICO']],
                                agrupado_piezas_RAF[['SUFIJO','PREFIJO','PIEC_DESCRIPTION','CANTIDAD','BASICO']]],
                               axis = 0)
# Agrupar por PBS, las piezas que estan en RAF saldran como cantidades negativas
piezas_agrupadas_PBS =piezas_sin_agrupar.groupby(['PREFIJO', 'BASICO', 'SUFIJO', 'PIEC_DESCRIPTION'])['CANTIDAD'].sum().reset_index()
# Agrupar por Basico
piezas_agrupadas_RAF = piezas_agrupadas_PBS.groupby(['BASICO'])['CANTIDAD'].sum().reset_index()
# Eliminar las piezas con cantidad 0 ya que se cancelaron con las negatigas de RAF
piezas_agrupadas_RAF = piezas_agrupadas_RAF.drop(piezas_agrupadas_RAF[piezas_agrupadas_RAF['CANTIDAD']==0.00].index)
# Sumar todos los negativos (cantidad de piezas RAF que existen aun y hay que sacar)
sum(i for i in piezas_agrupadas_RAF['CANTIDAD'] if i < 0)

-57936.0

In [72]:
piezas_sin_agrupar

,SUFIJO,PREFIJO,PIEC_DESCRIPTION,CANTIDAD,BASICO
0,A,022,"CUBIERTA, DE PLASTICO",1914.0,103534
1,AM,038,,957.0,903315
2,F,03C,,957.0,906051
3,,03G,POLEA LISA CON RODAMIENTO INTERIOR,957.0,145276
4,,03G,CAPUCHON DE CIERRE,957.0,145291
...,...,...,...,...,...
1216,,WHT,TORNILLO,-734.0,005599
1217,,WHT,Tornillo Combinado,-3696.0,006148
1218,,WHT,Tornillo Combinado,-4.0,006152
1219,,WHT,,-5736.0,007077


In [15]:
# Importar equivalencias 
equivalencias = pd.read_excel(r'C:\Volkswagen\Equivalencias.xlsx')
# Universo de Piezas a eliminar
piezas_eliminar = piezas_agrupadas_RAF[piezas_agrupadas_RAF['CANTIDAD']<0]
# Hacer copia de piezas_agrupadas_RAF para modificarla
piezas_agrupadas = piezas_agrupadas_RAF.copy()

In [16]:
# Controles
borrados = []
valores_borrados = 0
iguales = []
mayores = []
menores = []

In [17]:
for x in piezas_eliminar.itertuples():
    d = x[1]
    v = x[2]
# Buscar la equivalencia
    a = pd.merge(left=equivalencias[equivalencias['Basico']== str(d)], right= piezas_agrupadas[['BASICO','CANTIDAD']],
                 how='inner', left_on='Basico_EQ', right_on='BASICO')
# Borrar el Basico con valor negativo de las piezas agrupadas
    piezas_agrupadas = piezas_agrupadas.drop(piezas_agrupadas[piezas_agrupadas['BASICO']== d].index)
# Si en la equivalencica solo hay cantidades negativas o no hay valores, borrar el basico del negativo buscado
    if sum(i for i in a['CANTIDAD'] if i > 0) ==0 | len(a) == 0:
        borrados.append(d)
        valores_borrados += v
# Si existe por lo menos un valor absoluto de la cantidad buscada, modificar la cantidad del primer basico encontrado
    elif len(a[a['CANTIDAD']==abs(v)].reset_index()) >= 1:
        iguales.append(d)
        b = a[a['CANTIDAD']==abs(v)].reset_index()['BASICO'][0]
        piezas_agrupadas.loc[piezas_agrupadas['BASICO']== b,'CANTIDAD'] = piezas_agrupadas['CANTIDAD'] - abs(v)
# Si existen cantidades mayores al valor buscado, modificar la cantidad del basico cuyo valor sea el menor
    elif len(a[a['CANTIDAD']>abs(v)].reset_index()) >= 1:
        mayores.append(d)
        b = a[a['CANTIDAD']>abs(v)].sort_values(by = 'CANTIDAD').reset_index()['BASICO'][0]
        piezas_agrupadas.loc[piezas_agrupadas['BASICO']== b,'CANTIDAD'] = piezas_agrupadas['CANTIDAD'] - abs(v)
# Si no hay cantidades mayores al valor buscado pero la suma de los positivos es mayor al valor buscado
    elif sum(i for i in a['CANTIDAD'] if i > 0) >= abs(v):
        menores.append(d)
        for y in a[a['CANTIDAD']>0].sort_values(by = 'CANTIDAD', ascending = False).itertuples():
            v += y[9]
            if v<0:
                piezas_agrupadas.loc[piezas_agrupadas['BASICO']== str(y[8]),'CANTIDAD'] = 0
            else:
                piezas_agrupadas.loc[piezas_agrupadas['BASICO']== str(y[8]),'CANTIDAD'] = v
                break
    else:
        borrados.append(d)
        valores_borrados += v

In [18]:
# Comprovar que la cantidad de Basicos eliminados sea el correcto
len(borrados+iguales+mayores+menores) == len(piezas_eliminar)

True

In [19]:
# Comprovar que la cantidad de piezas eliminadas sean las correctas
sum(piezas_agrupadas['CANTIDAD'])+valores_borrados == sum(i for i in piezas_agrupadas_RAF['CANTIDAD'] if i > 0) + sum(i for i in piezas_agrupadas_RAF['CANTIDAD'] if i < 0)

True

In [20]:
# Limpiar los registros con cantidad 0 (ya que se suprimieron en la fase anterior)
piezas_agrupadas = piezas_agrupadas.drop(piezas_agrupadas[piezas_agrupadas['CANTIDAD']==0.00].index)

### Facturas productivas

In [21]:
# Cabecera
cab = ['SUFIJO','PREFIJO','PRECIO_NET','PERIODO_DD','NRO_VIAJE','NRO_PROV','NRO_DESPAC','NRO727','MANUAL','IVA_FACT',
       'IMPIVA_UNI','FACPROV_NS','FACPROV_NN','FACPROV_MM','FACPROV_DD','FACPROV_AA','DESCRIPCIO','DENOMIN_PR',
       'CUIT_PROV','COD_DOC','CANTIDAD','CANT_ASIGN','BASICO']

In [22]:
# Extraer el ultimo del historico facturas de compra
facturas_prod = pd.read_csv(r'C:\Volkswagen\Año de recupero 2017\Bienes Enero\02 Asignaciones\02 Eliminar comprobantes viejos\03 Fact a utilizar\facturas.csv',
                            sep= '\t' , encoding = 'unicode_escape', header= None, names = cab, 
                            converters={'FACPROV_NN': lambda x: str(x), 'FACPROV_NS': lambda x: str(x),
                                        'COD_DOC': lambda x: str(x),'CUIT_PROV': lambda x: str(x)})
# Eliminar las filas con 0 en la fecha
facturas_prod = facturas_prod.drop(facturas_prod[facturas_prod['PERIODO_DD']== 0].index)
# Ordenar por las mas antiguas
facturas_prod = facturas_prod.sort_values('PERIODO_DD')
# Comprobar que el campo cant_asig sea cero
facturas_prod[facturas_prod['CANT_ASIGN']!=0]

,SUFIJO,PREFIJO,PRECIO_NET,PERIODO_DD,NRO_VIAJE,NRO_PROV,NRO_DESPAC,NRO727,MANUAL,IVA_FACT,...,FACPROV_MM,FACPROV_DD,FACPROV_AA,DESCRIPCIO,DENOMIN_PR,CUIT_PROV,COD_DOC,CANTIDAD,CANT_ASIGN,BASICO


In [23]:
# Concatenar las columnas fecha y numero de factura
facturas_prod['FECHA_FACT'] = facturas_prod['FACPROV_DD'].astype(str)+'/'+facturas_prod['FACPROV_MM'].astype(str)+'/'+facturas_prod['FACPROV_AA'].astype(str)
facturas_prod['NRO_FACT']= facturas_prod['FACPROV_NN']+facturas_prod['FACPROV_NS']
facturas_prod['Index'] = facturas_prod.index

In [24]:
# Estructura de facturas_productivas
facturas_prod = facturas_prod[['FECHA_FACT','COD_DOC','PERIODO_DD','CUIT_PROV','NRO_PROV','DENOMIN_PR','NRO_FACT',
                               'PRECIO_NET','IVA_FACT','DESCRIPCIO','BASICO','PREFIJO','SUFIJO','CANTIDAD',
                               'CANT_ASIGN','IMPIVA_UNI', 'Index']]

### Asignacion

In [25]:
# Estructura de piezas asignadas
asignacion = pd.DataFrame({'FECHA_FACT':[],'COD_DOC':[],'PERIODO_DD':[],'CUIT_PROV':[],'NRO_PROV':[],'DENOMIN_PR':[],
                           'NRO_FACT':[],'PRECIO_NET':[],'IVA_FACT':[],'DESCRIPCIO':[], 'BASICO':[],
                           'PREFIJO':[],'SUFIJO':[],'CANTIDAD':[],'CANT_ASIGN':[],'IMPIVA_UNI':[], 'Index':[]})
# Copia de las piezas agrupadas
piezas_exportadas = piezas_agrupadas.copy()
# Preparar archivo de piezas
piezas_exportadas['CANT_ASIG']= 0
piezas_exportadas['CANT_PEND']= piezas_agrupadas['CANTIDAD']
# Para contar los basicos no asignados
count = 0

In [26]:
for x in piezas_exportadas.itertuples():
    basico = x[1]
    asig = x[3]
    pend = x[4]
# Buscar en el universo de facturas productivas el basico de piezas agrupadas
    universo = facturas_prod[facturas_prod['BASICO']== basico]
# Si no se encuentra el basico en facturas productivas
    if len(universo) == 0:
        # Buscar equivalencia
        a = pd.merge(left=equivalencias[['Basico','Basico_EQ']][equivalencias['Basico']== basico], 
                     right= facturas_prod[['BASICO','CANTIDAD','Index']], how='inner', 
                     left_on='Basico_EQ', right_on='BASICO')
        # Si no existe el basico en equivalencias o no hay ninguna equivalencia encontrada en facturas_prod
        if len(a) == 0:
            count += 1
        # Si hay equivalencias encontradas que coincidan con basicos en el universo de facturas productivas
        else:
            print(1)
            for i in a.itertuples():
                # Itero cada factura conincidente con la equivalencia hasta que ya no tenga piezas pendientes
                pend -= i[4]
                if pend > 0:
                    facturas_prod.loc[facturas_prod.index == i[5],'CANT_ASIGN'] = i[4]
                    piezas_exportadas.loc[piezas_exportadas['BASICO'] == basico,'CANT_ASIG'] = piezas_exportadas['CANT_ASIG'] + i[4]
                    piezas_exportadas.loc[piezas_exportadas['BASICO'] == basico,'CANT_PEND'] = pend
                    asignacion = pd.concat([asignacion, facturas_prod[facturas_prod.index == i[5]]],
                                sort=False,axis=0)
                else:
                    facturas_prod.loc[facturas_prod.index == i[5],'CANT_ASIGN'] = i[4] - abs(pend)
                    piezas_exportadas.loc[piezas_exportadas['BASICO'] == basico,'CANT_ASIG'] = piezas_exportadas['CANT_ASIG'] + i[4] - abs(pend)
                    piezas_exportadas.loc[piezas_exportadas['BASICO'] == basico,'CANT_PEND'] = 0
                    asignacion = pd.concat([asignacion, facturas_prod[facturas_prod.index == i[5]]],
                                sort=False,axis=0)  
                    break
# Si existen facturas productivas para asignar
    else:
        for y in universo.itertuples():
            # Itero cada factura hasta que ya no tenga piezas pendientes
            pend -= y[14]
            if pend > 0:
                facturas_prod.loc[facturas_prod.index == y[0],'CANT_ASIGN'] = y[14]
                piezas_exportadas.loc[piezas_exportadas['BASICO'] == basico,'CANT_ASIG'] = piezas_exportadas['CANT_ASIG'] + y[14]
                piezas_exportadas.loc[piezas_exportadas['BASICO'] == basico,'CANT_PEND'] = pend
                asignacion = pd.concat([asignacion, facturas_prod[facturas_prod.index == y[0]]],
                            sort=False,axis=0)
            else:
                facturas_prod.loc[facturas_prod.index == y[0],'CANT_ASIGN'] = y[14] - abs(pend)
                piezas_exportadas.loc[piezas_exportadas['BASICO'] == basico,'CANT_ASIG'] = piezas_exportadas['CANT_ASIG'] + y[14] - abs(pend)
                piezas_exportadas.loc[piezas_exportadas['BASICO'] == basico,'CANT_PEND'] = 0
                asignacion = pd.concat([asignacion, facturas_prod[facturas_prod.index == y[0]]],
                            sort=False,axis=0)  
                break

In [27]:
# Calcular el IVA computado, en la tabla piezas_asignadas (Cantidad asignada por Impiva_Uni)
asignacion['IVA_COMPUT']= asignacion['CANT_ASIGN']*asignacion['IMPIVA_UNI']
# Eliminar la columna Cantidad que viene de factura
asignacion = asignacion.drop(['CANTIDAD'], axis=1)

In [67]:
facturas_prod

,FECHA_FACT,COD_DOC,PERIODO_DD,CUIT_PROV,NRO_PROV,DENOMIN_PR,NRO_FACT,PRECIO_NET,IVA_FACT,DESCRIPCIO,BASICO,PREFIJO,SUFIJO,CANTIDAD,CANT_ASIGN,IMPIVA_UNI,Index,Eliminar,CLAVE
0,2/3/16,01,201603,30712029656,48285,METALURGICA MAIPU S.R.L.,000300000032,98993.81,20788.70,Bien Productivo,409169,02T,A0S02,1093,0.0,1.6054,0,NaN,0003000000323071202965601
5623,21/3/16,01,201603,30685076248,42107,TREVES ARGENTINA S.A.,000400004483,342311.84,71885.49,Bien Productivo,863955,5Z0,NaN,240,0.0,12.6500,5623,NaN,0004000044833068507624801
5622,21/3/16,01,201603,30685076248,42107,TREVES ARGENTINA S.A.,000400004482,71214.00,14954.94,Bien Productivo,863463,5Z9,RR6,65,0.0,28.7590,5622,NaN,0004000044823068507624801
5621,21/3/16,01,201603,30685076248,42107,TREVES ARGENTINA S.A.,000400004482,71214.00,14954.94,Bien Productivo,863463,5Z9,RR6,195,0.0,28.7590,5621,NaN,0004000044823068507624801
5620,21/3/16,01,201603,30685076248,42107,TREVES ARGENTINA S.A.,000400004482,71214.00,14954.94,Bien Productivo,863463,5Z9,RR6,195,0.0,28.7590,5620,NaN,0004000044823068507624801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826133,13/1/20,01,202001,33712130399,48415,PLASTIC OMNIUM AUTOMOTIVE ARGE,000900000083,12762413.32,2680106.91,Bien Productivo,805904,2H6,B9B9,1,0.0,6.5400,826133,NaN,0009000000833371213039901
826132,13/1/20,01,202001,33712130399,48415,PLASTIC OMNIUM AUTOMOTIVE ARGE,000900000083,12762413.32,2680106.91,Bien Productivo,853728,2H6,C9X,33,0.0,65.5148,826132,NaN,0009000000833371213039901
826131,13/1/20,01,202001,33712130399,48415,PLASTIC OMNIUM AUTOMOTIVE ARGE,000900000083,12762413.32,2680106.91,Bien Productivo,853728,2H6,E7Q,44,0.0,65.5148,826131,NaN,0009000000833371213039901
826142,13/1/20,01,202001,33712130399,48415,PLASTIC OMNIUM AUTOMOTIVE ARGE,000900000083,12762413.32,2680106.91,Bien Productivo,853717,2HH,E7Q,44,0.0,343.7950,826142,NaN,0009000000833371213039901


In [28]:
piezas_no_asign = pd.merge(left=piezas_exportadas[piezas_exportadas['CANT_ASIG'] == 0], 
                     right= piezas_agrupadas_PBS.drop_duplicates(['BASICO'], keep= 'first')[['BASICO','PREFIJO','SUFIJO','PIEC_DESCRIPTION']], how='inner', 
                     left_on='BASICO', right_on='BASICO')

In [66]:
piezas_no_asign.to_excel('piezas_no_asign.xlsx')

In [29]:
asignacion.groupby(['BASICO'])['CANT_ASIGN'].sum().reset_index()

,BASICO,CANT_ASIGN
0,000002,2198.0
1,000007,955.0
2,000009,318.0
3,000010,518.0
4,000011,519.0
...,...,...
697,971369,922.0
698,971739,2873.0
699,971830,461.0
700,971882,461.0


In [30]:
# Basicos analizados de piezas exportadas (debe dar True)
len(piezas_exportadas)-len(asignacion.groupby(['BASICO'])['CANT_ASIGN'].sum().reset_index())-len(piezas_no_asign)

0

In [70]:
len(piezas_exportadas)

1150

In [31]:
# Piezas que faltaron
sum(piezas_exportadas['CANTIDAD'])-sum(asignacion['CANT_ASIGN'])

257690.29300000018

In [32]:
# IVA Computable
sum(asignacion['IVA_COMPUT'])

35791351.43230274

### Actualizar el universo de Factudas Productivas 

Temas pendientes:
* Tema de seguimiento de las facturas
* Seguir el proceso de Bienes Capital
* Exportar los archivos de respaldo

Verificaciones:
* Mirar si hay duplicados en equivalencia
* Mirar si las piezas no asignadas existe algun basico en el universo de facturas original


In [33]:
# Comprobantes asignados, parcial o totalmente (las filas deben ser igual a la asignacion)
facturas_prod[facturas_prod['CANT_ASIGN'] != 0.00]

,FECHA_FACT,COD_DOC,PERIODO_DD,CUIT_PROV,NRO_PROV,DENOMIN_PR,NRO_FACT,PRECIO_NET,IVA_FACT,DESCRIPCIO,BASICO,PREFIJO,SUFIJO,CANTIDAD,CANT_ASIGN,IMPIVA_UNI,Index
5597,23/3/16,01,201603,30505002845,16145,CIBIE ARGENTINA S.A.,001200004416,4124418.40,866127.80,Bien Productivo,945096,2H2,E,63,63.0,76.0210,5597
5596,23/3/16,01,201603,30505002845,16145,CIBIE ARGENTINA S.A.,001200004416,4124418.40,866127.80,Bien Productivo,945095,2H2,E,315,315.0,69.9610,5596
5595,23/3/16,01,201603,30505002845,16145,CIBIE ARGENTINA S.A.,001200004416,4124418.40,866127.80,Bien Productivo,945096,2H2,E,84,84.0,76.2780,5595
5594,23/3/16,01,201603,30505002845,16145,CIBIE ARGENTINA S.A.,001200004416,4124418.40,866127.80,Bien Productivo,945095,2H2,E,336,336.0,72.6550,5594
5593,23/3/16,01,201603,30505002845,16145,CIBIE ARGENTINA S.A.,001200004416,4124418.40,866127.80,Bien Productivo,945096,2H2,E,84,84.0,78.9490,5593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824415,28/1/20,01,202001,30711162085,47220,KROMBERG & SCHUBERT ARGENTINA,000900000053,31033162.08,6516964.04,Bien Productivo,970004,2H0,AD,32,32.0,15.2078,824415
824414,28/1/20,01,202001,30711162085,47220,KROMBERG & SCHUBERT ARGENTINA,000900000053,31033162.08,6516964.04,Bien Productivo,970004,2H0,AC,54,54.0,18.7270,824414
824408,28/1/20,01,202001,30711162085,47220,KROMBERG & SCHUBERT ARGENTINA,000900000053,31033162.08,6516964.04,Bien Productivo,970220,2H0,A,160,160.0,43.6127,824408
824407,28/1/20,01,202001,30711162085,47220,KROMBERG & SCHUBERT ARGENTINA,000900000053,31033162.08,6516964.04,Bien Productivo,970225,2H0,L,179,114.0,29.6617,824407


In [34]:
# Facturas productivas a sacar, eliminar = 1
facturas_prod.loc[facturas_prod['CANTIDAD']-facturas_prod['CANT_ASIGN']== 0, 'Eliminar'] = 1

In [35]:
# Quitar columnas que estan de mas
asignacion = asignacion.drop(['Index'], axis= 1)
end_time = time() - start_time

### Separar Nacionales de importadas

In [36]:
# Facturas importadas (son las que tiene COD_DOC = 14 pero no se encuentra ninguno)
facturas_prod[facturas_prod['COD_DOC']==14]

,FECHA_FACT,COD_DOC,PERIODO_DD,CUIT_PROV,NRO_PROV,DENOMIN_PR,NRO_FACT,PRECIO_NET,IVA_FACT,DESCRIPCIO,BASICO,PREFIJO,SUFIJO,CANTIDAD,CANT_ASIGN,IMPIVA_UNI,Index,Eliminar


In [37]:
# Agrupar la asignacion
asignacion_agrupada = asignacion.groupby(['FECHA_FACT','COD_DOC','PERIODO_DD','CUIT_PROV','NRO_PROV','DENOMIN_PR',
                                          'NRO_FACT','DESCRIPCIO','PRECIO_NET','IVA_FACT',
                                          ])['IVA_COMPUT'].sum().reset_index()
# Agregar columna de marca productiva, Amarok
asignacion_agrupada['MARCA_PROD']='AM'

In [38]:
# Comprobar que todos los IVA computados son menores al IVA facturado
asignacion_agrupada.loc[asignacion_agrupada['IVA_FACT']>=asignacion_agrupada['IVA_FACT'],'Ok']=1
sum(asignacion_agrupada['Ok'])==len(asignacion_agrupada)

True

In [39]:
asignacion_agrupada['CLAVE']= asignacion_agrupada['NRO_FACT']+asignacion_agrupada['CUIT_PROV']+asignacion_agrupada['COD_DOC']

In [40]:
facturas_prod['CLAVE']= facturas_prod['NRO_FACT']+facturas_prod['CUIT_PROV']+facturas_prod['COD_DOC']

In [41]:
asignacion_agrupada['FirstOfMotivo no retención']= 0

In [42]:
asignacion_agrupada[asignacion_agrupada.duplicated(['CLAVE'])]

,FECHA_FACT,COD_DOC,PERIODO_DD,CUIT_PROV,NRO_PROV,DENOMIN_PR,NRO_FACT,DESCRIPCIO,PRECIO_NET,IVA_FACT,IVA_COMPUT,MARCA_PROD,Ok,CLAVE,FirstOfMotivo no retención


In [69]:
asignacion_agrupada.to_excel('asignacion_agrupada.xlsx')

### Control de Excesos 

Control de excesos
   * Saca las claves que fueron utilizadas en un reproceso anterior

Correccion automatica
   * Buscar para que sirve

Queda pendiente que voy a hacer con las facturas!
   * Revisar de los reprocesos anteriores que si las facturas se quitan o que se hace
   * Siempre uso el mismo universo de facturas
   * Falta hacer lo de preparar el archivo de facturas a ver si se me escapa algo

In [43]:
end_time/60

8.619842946529388

In [44]:
facturas_prod[facturas_prod['CLAVE']=='0002000025453070940734801']

,FECHA_FACT,COD_DOC,PERIODO_DD,CUIT_PROV,NRO_PROV,DENOMIN_PR,NRO_FACT,PRECIO_NET,IVA_FACT,DESCRIPCIO,BASICO,PREFIJO,SUFIJO,CANTIDAD,CANT_ASIGN,IMPIVA_UNI,Index,Eliminar,CLAVE
206544,17/8/17,01,201709,30709407348,44412,METALURGICA MERESHIAN S.R.L,000200002545,400010.4,84002.18,Bien Productivo,311159,02T,P0S00,864,0.0,2.5647,206544,NaN,0002000025453070940734801
206545,17/8/17,01,201709,30709407348,44412,METALURGICA MERESHIAN S.R.L,000200002545,400010.4,84002.18,Bien Productivo,311531,02A,Q0S00,972,0.0,1.7010,206545,NaN,0002000025453070940734801
206546,17/8/17,01,201709,30709407348,44412,METALURGICA MERESHIAN S.R.L,000200002545,400010.4,84002.18,Bien Productivo,311159,02T,P0S00,864,0.0,2.5681,206546,NaN,0002000025453070940734801
206547,17/8/17,01,201709,30709407348,44412,METALURGICA MERESHIAN S.R.L,000200002545,400010.4,84002.18,Bien Productivo,311531,02A,Q0S00,639,0.0,1.7010,206547,NaN,0002000025453070940734801
206548,17/8/17,01,201709,30709407348,44412,METALURGICA MERESHIAN S.R.L,000200002545,400010.4,84002.18,Bien Productivo,311159,02T,P0S00,864,0.0,2.5647,206548,NaN,0002000025453070940734801
206549,17/8/17,01,201709,30709407348,44412,METALURGICA MERESHIAN S.R.L,000200002545,400010.4,84002.18,Bien Productivo,311159,02T,P0S00,432,0.0,2.5662,206549,NaN,0002000025453070940734801
206550,17/8/17,01,201709,30709407348,44412,METALURGICA MERESHIAN S.R.L,000200002545,400010.4,84002.18,Bien Productivo,311361,02Z,F0S00,1444,0.0,1.7010,206550,NaN,0002000025453070940734801
206551,17/8/17,01,201709,30709407348,44412,METALURGICA MERESHIAN S.R.L,000200002545,400010.4,84002.18,Bien Productivo,311159,02T,P0S00,864,0.0,2.5662,206551,NaN,0002000025453070940734801
206552,17/8/17,01,201709,30709407348,44412,METALURGICA MERESHIAN S.R.L,000200002545,400010.4,84002.18,Bien Productivo,311361,02Z,F0S00,1083,0.0,1.7010,206552,NaN,0002000025453070940734801


### Compras Nacionales

In [45]:
compras_productivas = pd.read_csv(r'C:\Users\llloveras002\Documents\Compras_productivas_Final.txt', sep= '\t',
                                 converters={'Nro_PROV': lambda x: str(x),'COD_DOC': lambda x: str(x)})

In [46]:
compras = pd.read_excel('C:\clientes\Volkswagen Nuevo\Compras Nacionales\Febrero\Compras Mes Final.xlsx',
                       converters={'Número_Comprobante': lambda x: str(x),'Tipo_Comprobante': lambda x: str(x)})

In [47]:
proveedores = pd.read_excel('C:\clientes\Volkswagen Nuevo\Compras Nacionales\Febrero\Proveedores.xls')

In [48]:
proveedores['CUIT']=proveedores['Nº ident.fis.1']

In [49]:
proveedores['NRO_PROV']= proveedores['Acreedor'].astype(str).str[3:]

In [50]:
compras_productivas_cuit = pd.merge(left= compras_productivas, right= proveedores[['NRO_PROV','CUIT']], how='inner', 
                     left_on='Nro_PROV', right_on='NRO_PROV')

In [51]:
compras_productivas_cuit.loc[compras_productivas_cuit['COD_DOC']=='09', 'CLAVE'] = compras_productivas_cuit['FACPROV_NRON']+ compras_productivas_cuit['CUIT']+'01'

In [52]:
compras_productivas_cuit['CLAVE']=compras_productivas_cuit['CLAVE'].fillna(compras_productivas_cuit['FACPROV_NRON']+ compras_productivas_cuit['CUIT']+'02')

In [98]:
compras['CLAVE']= compras['Número_Comprobante'].str[12:]+compras['ID_Vendedor'].astype(str)+compras['Tipo_Comprobante'].str[1:]

In [103]:
compras['CUIT']= compras['ID_Vendedor'].astype(str)

In [54]:
facturas_productivas = pd.merge(left= compras_productivas_cuit, right= compras, how='inner', 
                     left_on='CLAVE', right_on='CLAVE')

In [55]:
facturas_productivas['Precio_Neto']=facturas_productivas['Neto_gravado']/100

In [56]:
facturas_productivas['CUIT_Proveedor']= facturas_productivas['ID_Vendedor']

In [57]:
facturas_productivas['IVA_Fact']= facturas_productivas['Crédito_Fiscal_Computable']/100

In [58]:
facturas_productivas['Periodo_DDJJ'] = '202001'

In [73]:
facturas_productivas[['Nro_PROV', 'COD_DOC', 'IMPIVA_UNIT', 'Cantidad', 'FACPROV_NROL',
                                          'FACPROV_NRON', 'BASICO', 'PREFIJO', 'SUFIJO', 'CUIT_Proveedor',
                                          'Denominación_Vendedor', 'Precio_Neto', 'IVA_Fact', 'FACPROV_AA',
                                          'FACPROV_MM', 'FACPROV_DD', 'Periodo_DDJJ']]

,Nro_PROV,COD_DOC,IMPIVA_UNIT,Cantidad,FACPROV_NROL,FACPROV_NRON,BASICO,PREFIJO,SUFIJO,CUIT_Proveedor,Denominación_Vendedor,Precio_Neto,IVA_Fact,FACPROV_AA,FACPROV_MM,FACPROV_DD,Periodo_DDJJ
0,02442,09,"0,01",291,22,00003075,881021,ASI,C,30635530665,PABSA SA,19251361.67,4042785.95,20,1,14,202001
1,02442,09,"13,66",307,22,00003075,000080,L2H,,30635530665,PABSA SA,19251361.67,4042785.95,20,1,14,202001
2,02442,09,"0,01",15,22,00003075,881021,ASI,D,30635530665,PABSA SA,19251361.67,4042785.95,20,1,14,202001
3,02442,09,"590,32",215,22,00003075,881105,2K0,G,30635530665,PABSA SA,19251361.67,4042785.95,20,1,14,202001
4,02442,09,"31,29",10,22,00003075,963553,2K5,,30635530665,PABSA SA,19251361.67,4042785.95,20,1,14,202001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7012,53041,09,"25,21",84,3,00000024,301107,0A4,ANS12,30712292950,IMMAUTOPARTES SA,1452967.53,305123.18,19,12,23,202001
7013,53041,09,"4,97",840,3,00000024,311244,02J,00S01,30712292950,IMMAUTOPARTES SA,1452967.53,305123.18,19,12,23,202001
7014,53041,09,"4,87",506,3,00000024,311251,02J,Q0S10,30712292950,IMMAUTOPARTES SA,1452967.53,305123.18,19,12,23,202001
7015,53041,09,"17,34",150,3,00000024,409156,0A4,F0S20,30712292950,IMMAUTOPARTES SA,1452967.53,305123.18,19,12,23,202001


In [104]:
df = compras.merge(compras_productivas_cuit, how='outer', indicator=True)

In [105]:
servicios = df[df['_merge'] == "left_only"].drop('_merge', 1)

In [106]:
len(compras)

3901

In [107]:
len(servicios)

3739

In [115]:
servicios_sinff = servicios[~servicios.CUIT.isin(['30707962743','30519071521','00000000000'])]

In [125]:
concesionarios = pd.read_excel('C:/Volkswagen/Concesionarios y Publicidad.xlsx', sheet_name = 'Concesionarios')

In [128]:
concesionarios['CUIT'] = concesionarios['CUIT'].astype(str)

In [134]:
publicidad = pd.read_excel('C:/Volkswagen/Concesionarios y Publicidad.xlsx', sheet_name = 'Publicidad')

In [129]:
df1 = servicios_sinff.merge(concesionarios, how='outer', indicator=True)

In [139]:
servicios_sinff_c = df1[df1['_merge'] == "left_only"].drop('_merge', 1)

In [140]:
len(servicios_sinff_c)

3147

In [149]:
len(publicidad)

45

In [159]:
publicidad['CUIT'] = publicidad['CUIT'].astype(str)

In [195]:
df2 = pd.merge(left= servicios_sinff_c, right= publicidad, how='outer', 
                     left_on='CUIT', right_on='CUIT', indicator = True)

In [197]:
servicios_sinff_cp = df2[df2['_merge'] == "left_only"].drop('_merge', 1)

In [199]:
len(servicios_sinff_cp)

3074

In [219]:
servicios_sinff_cp40 = servicios_sinff_cp[servicios_sinff_cp['Crédito_Fiscal_Computable']/100>40]

In [244]:
len(servicios_sinff_cp40)

2764

In [245]:
servicios_sinff_cp40['IVA'] = servicios_sinff_cp40['Crédito_Fiscal_Computable']/100

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [243]:
len(proveedores)-len(proveedores_sin_duplicados)

340

In [237]:
proveedores[proveedores.duplicated(['CUIT'])].sort_values(by=['CUIT'])

,Soc.,Acreedor,Nombre 1,Nº ident.fis.1,CUIT,NRO_PROV
5607,NaN,80144934,QUALITY MARKET S.A.,-,-,44934
9786,NaN,80153300,AHUMADA LUIS EDUARDO - BLOQUEADO,20049992883,20049992883,53300
1083,NaN,80129384,VILLALON RAFAEL,20064189914,20064189914,29384
6829,NaN,80147072,MAYORGA PABLO TOMAS,20085503538,20085503538,47072
8110,NaN,80148388,RIVERA JUAN FERNANDO,20132209155,20132209155,48388
...,...,...,...,...,...,...
2247,NaN,80141475,RAMOS MEJIA CARLOS,X,X,41475
1893,NaN,80140879,FRANCKE CARLOS DANIEL,X,X,40879
5207,NaN,80144531,MENZO LUIS ANTONIO,X,X,44531
3057,NaN,80142340,MAYORGA PABLO T.,X,X,42340


In [239]:
proveedores_sin_duplicados = proveedores.drop_duplicates(['CUIT'], keep= 'first')

In [247]:
servicios_sin_formato = pd.merge(left= servicios_sinff_cp40, right= proveedores_sin_duplicados, how='inner', 
                     left_on='CUIT', right_on='CUIT', indicator = True)

In [ ]:
Quede en servicios final de facturas que no cruzan en el acl